<a href="https://colab.research.google.com/github/nithinag10/speech_command_recognition/blob/main/correct_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!unzip updated_data_1.zip

Archive:  updated_data_1.zip
   creating: updated_data_1/
  inflating: updated_data_1/.DS_Store  
  inflating: __MACOSX/updated_data_1/._.DS_Store  
   creating: updated_data_1/sakre/
   creating: updated_data_1/half/
   creating: updated_data_1/uppu/
   creating: updated_data_1/one/
  inflating: updated_data_1/sakre/.DS_Store  
  inflating: __MACOSX/updated_data_1/sakre/._.DS_Store  
  inflating: updated_data_1/sakre/Recording_76.wav  
  inflating: __MACOSX/updated_data_1/sakre/._Recording_76.wav  
  inflating: updated_data_1/sakre/Recording_89.wav  
  inflating: __MACOSX/updated_data_1/sakre/._Recording_89.wav  
  inflating: updated_data_1/sakre/Record (online-voice-recorder.com)-15.wav  
  inflating: __MACOSX/updated_data_1/sakre/._Record (online-voice-recorder.com)-15.wav  
  inflating: updated_data_1/sakre/Record (online-voice-recorder.com)-14.wav  
  inflating: __MACOSX/updated_data_1/sakre/._Record (online-voice-recorder.com)-14.wav  
  inflating: updated_data_1/sakre/Recording_

In [ ]:
!pip install torchaudio

     |████████████████████████████████| 1.9MB 4.1MB/s 


In [ ]:
! pip install comet_ml
# import comet_ml at the top of your file
from comet_ml import Experiment

# Create an experiment with your api key
experiment = Experiment(
    api_key="LwD999aUpPlN5q9K1toSt9Mq5",
    project_name="speech-command",
    workspace="nithinag10",
)

     |████████████████████████████████| 276kB 4.3MB/s 
     |████████████████████████████████| 71kB 8.5MB/s 
     |████████████████████████████████| 61kB 6.8MB/s 
     |████████████████████████████████| 532kB 32.8MB/s 
  Created wheel for configobj: filename=configobj-5.0.6-cp37-none-any.whl size=34547 sha256=cb5a01d115dd6ad8c41b0680719b2d72abcb1ebc420b6c8feb56dd36ba4885af
  Stored in directory: /root/.cache/pip/wheels/f1/e4/16/4981ca97c2d65106b49861e0b35e2660695be7219a2d351ee0
Successfully built configobj


COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Experiment is live on comet.ml https://www.comet.ml/nithinag10/speech-command/8848072c0418489fa328812438808dc0



In [ ]:

import os
import torchaudio
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchaudio
import matplotlib.pyplot as plt
import IPython.display as ipd
from tqdm.notebook import tqdm

In [ ]:
def data_augment1(tensor , rate):
  effects = [
  ["lowpass", "-1", "200"], # apply single-pole lowpass filter
  ["speed", "0.8"],  # reduce the speed
                     # This only changes sample rate, so it is necessary to
                     # add `rate` effect with original sample rate after this.
  ["rate", f"{rate}"],
 # Reverbration gives some dramatic feeling
]
  waveform, sample_rate = torchaudio.sox_effects.apply_effects_tensor(
    tensor, rate, effects)
  return waveform , sample_rate

def data_augment2(tensor , rate):
  effects = [
    ["lowpass", "-2", "200"], # apply single-pole lowpass filter
    ["speed", "1.2"],  # reduce the speed
                      # This only changes sample rate, so it is necessary to
                      # add `rate` effect with original sample rate after this.
    ["rate", f"{rate}"],
  ]
  waveform, sample_rate = torchaudio.sox_effects.apply_effects_tensor(
    tensor, rate, effects)
  return waveform , sample_rate



def read_data_from_directory(path):
  tensor_list = []
  label = []
  rate_list = []
  directory = path
  for filename in os.listdir(directory):
    if filename == ".DS_Store":
      continue
    else :
      for audio_file in os.listdir(directory + '/' + filename):
        if audio_file == ".DS_Store":
          continue
        else:
          tensor , rate = torchaudio.load(directory + '/' + filename + '/' + audio_file)
          tensor_list.append(tensor)
          rate_list.append(rate)
          label.append(filename)
          tensor_aug1 , rate_aug1 = data_augment1(tensor , rate)
          tensor_list.append(tensor_aug1)
          rate_list.append(rate_aug1)
          label.append(filename)
          tensor_aug2 , rate_aug2 = data_augment2(tensor , rate)
          tensor_list.append(tensor_aug2)
          rate_list.append(rate_aug2)
          label.append(filename)
  return tensor_list , label , rate_list



In [ ]:
class CustomAudioData:
  def __init__(self , audio_tensor , audio_label , audio_rate):
    self.tensor = audio_tensor
    self.label = audio_label
    self.rate = audio_rate

  def __len__(self):
    return len(self.label)

  def __getitem__(self , index):
    
    sample_tensor = self.tensor[index]
    sample_label = self.label[index]
    sample_rate = self.rate[index]

    return (sample_tensor , sample_rate , sample_label)

In [ ]:
tensor_list , label , rate_list = read_data_from_directory('updated_data_1')
dataset = CustomAudioData(tensor_list , label , rate_list)

In [ ]:
labels = sorted(list(set(datapoint[2] for datapoint in dataset)))
labels

['backward',
 'bed',
 'bird',
 'cat',
 'dog',
 'down',
 'eight',
 'five',
 'follow',
 'forward',
 'four',
 'go',
 'happy',
 'house',
 'learn',
 'left',
 'marvin',
 'nine',
 'no',
 'off',
 'on',
 'one',
 'right',
 'seven',
 'sheila',
 'six',
 'stop',
 'three',
 'tree',
 'two',
 'up',
 'visual',
 'wow',
 'yes',
 'zero']

In [ ]:
from torchaudio.datasets import SPEECHCOMMANDS
import os


class SubsetSC(SPEECHCOMMANDS):
    def __init__(self, subset: str = None):
        super().__init__("./", download=True)

        def load_list(filename):
            filepath = os.path.join(self._path, filename)
            with open(filepath) as fileobj:
                return [os.path.join(self._path, line.strip()) for line in fileobj]

        if subset == "validation":
            self._walker = load_list("validation_list.txt")
        elif subset == "testing":
            self._walker = load_list("testing_list.txt")
        elif subset == "training":
            excludes = load_list("validation_list.txt") + load_list("testing_list.txt")
            excludes = set(excludes)
            self._walker = [w for w in self._walker if w not in excludes]


# Create training and testing split of the data. We do not use validation in this tutorial.
dataset = SubsetSC("training")
# test_set = SubsetSC("testing")

waveform, sample_rate, label, speaker_id, utterance_number = dataset[0]

In [ ]:

waveform, sample_rate, label = dataset[0]
new_sample_rate = 8000
transform = torchaudio.transforms.Resample(orig_freq=sample_rate, new_freq=new_sample_rate)
transformed = transform(waveform)

print(transformed.shape)

ipd.Audio(transformed.numpy(), rate=new_sample_rate)

torch.Size([1, 8000])


In [ ]:
dataset[0]

(tensor([[-0.0658, -0.0709, -0.0753,  ..., -0.0700, -0.0731, -0.0704]]),
 16000,
 'backward',
 '0165e0e8',
 0)

In [ ]:
def label_to_index(word):
    # Return the position of the word in labels
    return torch.tensor(labels.index(word))


def index_to_label(index):
    # Return the word corresponding to the index in labels
    # This is the inverse of label_to_index
    return labels[index]


word_start = "bed"
index = label_to_index(word_start)
word_recovered = index_to_label(index)

print(word_start, "-->", index, "-->", word_recovered)



bed --> tensor(1) --> bed


In [ ]:
print(word_recovered)

bed


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def pad_sequence(batch):
    # Make all tensor in a batch the same length by padding with zeros
    batch = [item.t() for item in batch]
    batch = torch.nn.utils.rnn.pad_sequence(batch, batch_first=True, padding_value=0.)
    return batch.permute(0, 2, 1)


def collate_fn(batch):

    # A data tuple has the form:
    # waveform, sample_rate, label, speaker_id, utterance_number

    tensors, targets = [], []

    # Gather in lists, and encode labels as indices
    for waveform, _, label, *_ in batch:
        tensors += [waveform]
        targets += [label_to_index(label)]

    # Group the list of tensors into a batched tensor
    tensors = pad_sequence(tensors)
    targets = torch.stack(targets)

    return tensors, targets


batch_size = 15

if device == "cuda":
    num_workers = 1
    pin_memory = True
else:
    num_workers = 0
    pin_memory = False

train_loader = torch.utils.data.DataLoader(
    dataset,
    batch_size=batch_size,
    shuffle=True,
    collate_fn=collate_fn,
    num_workers=num_workers,
    pin_memory=pin_memory,
    drop_last = True
)

In [ ]:
device

device(type='cuda')

In [ ]:
class M5(nn.Module):
    def __init__(self, n_input, n_output , n_channel = 32):
        super().__init__()
        self.conv1 = nn.Conv1d(n_input, n_channel,kernel_size=80,  stride=4)
        self.bn1 = nn.BatchNorm1d(n_channel)
        self.pool1 = nn.MaxPool1d(4)
        self.conv2 = nn.Conv1d(n_channel, n_channel, kernel_size=3)
        self.bn2 = nn.BatchNorm1d(n_channel)
        self.pool2 = nn.MaxPool1d(4)
        self.conv3 = nn.Conv1d(n_channel, 2 * n_channel, kernel_size=3)
        self.bn3 = nn.BatchNorm1d(2 * n_channel)
        self.pool3 = nn.MaxPool1d(4)
        self.conv4 = nn.Conv1d(2 * n_channel, 4 * n_channel, kernel_size=3)
        self.bn4 = nn.BatchNorm1d(4 * n_channel)
        self.pool4 = nn.MaxPool1d(4)
        self.avgPool = nn.AvgPool1d(30) #input should be 512x30 so this outputs a 512x1
        self.fc1 = nn.Linear(4 * n_channel , n_output )

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(self.bn1(x))
        x = self.pool1(x)
        x = self.conv2(x)
        x = F.relu(self.bn2(x))
        x = self.pool2(x)
        x = self.conv3(x)
        x = F.relu(self.bn3(x))
        x = self.pool3(x)
        x = self.conv4(x)
        x = F.relu(self.bn4(x))
        x = self.pool4(x)
        x = F.avg_pool1d(x, x.shape[-1])
        x = x.permute(0, 2, 1)
        x = self.fc1(x)
        return F.log_softmax(x, dim=2)


model = M5(n_input=transformed.shape[0], n_output=len(labels))
model.to(device)

def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)


n = count_parameters(model)
print("Number of parameters: %s" % n)

Number of parameters: 41635


In [ ]:
optimizer = optim.Adam(model.parameters(), lr=0.01, weight_decay=0.0001)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.1)

In [ ]:
def train(model, epoch, log_interval):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):

        data = data.to(device)
        target = target.to(device)
        
        # apply transform and model on whole batch directly on device
        data = transform(data)
        output = model(data)

        # negative log-likelihood for a tensor of size (batch x 1 x n_output)
        loss = F.nll_loss(output.squeeze(), target)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # print training stats
        if batch_idx % log_interval == 0:
            print(f"Train Epoch: {epoch} [{batch_idx * len(data)}/{len(train_loader.dataset)} ({100. * batch_idx / len(train_loader):.0f}%)]\tLoss: {loss.item():.6f}")

        # update progress bar
        pbar.update(pbar_update)
        # record loss
        losses.append(loss.item())

In [ ]:
def number_of_correct(pred, target):
    # count number of correct predictions
    return pred.squeeze().eq(target).sum().item()


def get_likely_index(tensor):
    # find most likely label index for each element in the batch
    return tensor.argmax(dim=-1)

In [ ]:
log_interval = 20
n_epoch = 15

pbar_update = 1 / (len(train_loader))
losses = []

# The transform needs to live on the same device as the model and the data.
transform = transform.to(device)
with tqdm(total=n_epoch) as pbar:
    for epoch in range(1, n_epoch + 1):
        train(model, epoch, log_interval)
        scheduler.step()

Streaming output truncated to the last 5000 lines.
Train Epoch: 1 [88800/105829 (84%)]	Loss: 0.745522
Train Epoch: 1 [89100/105829 (84%)]	Loss: 0.800769
Train Epoch: 1 [89400/105829 (84%)]	Loss: 1.418595
Train Epoch: 1 [89700/105829 (85%)]	Loss: 0.879502
Train Epoch: 1 [90000/105829 (85%)]	Loss: 1.074310
Train Epoch: 1 [90300/105829 (85%)]	Loss: 1.337783
Train Epoch: 1 [90600/105829 (86%)]	Loss: 2.230295
Train Epoch: 1 [90900/105829 (86%)]	Loss: 1.108522
Train Epoch: 1 [91200/105829 (86%)]	Loss: 1.264734
Train Epoch: 1 [91500/105829 (86%)]	Loss: 1.114491
Train Epoch: 1 [91800/105829 (87%)]	Loss: 1.264805
Train Epoch: 1 [92100/105829 (87%)]	Loss: 0.486483
Train Epoch: 1 [92400/105829 (87%)]	Loss: 1.445719
Train Epoch: 1 [92700/105829 (88%)]	Loss: 1.212888
Train Epoch: 1 [93000/105829 (88%)]	Loss: 1.159271
Train Epoch: 1 [93300/105829 (88%)]	Loss: 1.033670
Train Epoch: 1 [93600/105829 (88%)]	Loss: 1.840257
Train Epoch: 1 [93900/105829 (89%)]	Loss: 0.558064
Train Epoch: 1 [94200/105829 (8

In [ ]:
def predict(tensor):
    # Use the model to predict the label of the waveform
    tensor = tensor.to(device)
    tensor = transform(tensor)
    tensor = model(tensor.unsqueeze(0))
    tensor = get_likely_index(tensor)
    tensor = index_to_label(tensor.squeeze())
    return tensor


waveform, sample_rate, utterance, *_ = dataset[50]
ipd.Audio(waveform.numpy(), rate=sample_rate)

print(f"Expected: {utterance}. Predicted: {predict(waveform)}.")



Expected: one. Predicted: half.


In [ ]:
torch.save(model.save_dict() , 'm5_model_trained_9.pth')

AttributeError: ignored

In [ ]:
from google.colab import output as colab_output
from base64 import b64decode
from io import BytesIO
from pydub import AudioSegment


RECORD = """
const sleep  = time => new Promise(resolve => setTimeout(resolve, time))
const b2text = blob => new Promise(resolve => {
  const reader = new FileReader()
  reader.onloadend = e => resolve(e.srcElement.result)
  reader.readAsDataURL(blob)
})
var record = time => new Promise(async resolve => {
  stream = await navigator.mediaDevices.getUserMedia({ audio: true })
  recorder = new MediaRecorder(stream)
  chunks = []
  recorder.ondataavailable = e => chunks.push(e.data)
  recorder.start()
  await sleep(time)
  recorder.onstop = async ()=>{
    blob = new Blob(chunks)
    text = await b2text(blob)
    resolve(text)
  }
  recorder.stop()
})
"""


def record(seconds=2):
    display(ipd.Javascript(RECORD))
    print(f"Recording started for {seconds} seconds.")
    s = colab_output.eval_js("record(%d)" % (seconds * 1000))
    print("Recording ended.")
    b = b64decode(s.split(",")[1])

    fileformat = "wav"
    filename = f"_audio.{fileformat}"
    AudioSegment.from_file(BytesIO(b)).export(filename, format=fileformat)
    return torchaudio.load(filename)


waveform, sample_rate = record()
print(f"Predicted: {predict(waveform)}.")
ipd.Audio(waveform.numpy(), rate=sample_rate)

<IPython.core.display.Javascript object>

Recording started for 2 seconds.
Recording ended.
Predicted: three.


In [ ]:
# transer learing
torch.save(model.state_dict() , 'm5_model_9.pth')

In [ ]:
model

M5(
  (conv1): Conv1d(1, 32, kernel_size=(80,), stride=(4,))
  (bn1): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool1): MaxPool1d(kernel_size=4, stride=4, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv1d(32, 32, kernel_size=(3,), stride=(1,))
  (bn2): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool2): MaxPool1d(kernel_size=4, stride=4, padding=0, dilation=1, ceil_mode=False)
  (conv3): Conv1d(32, 64, kernel_size=(3,), stride=(1,))
  (bn3): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool3): MaxPool1d(kernel_size=4, stride=4, padding=0, dilation=1, ceil_mode=False)
  (conv4): Conv1d(64, 128, kernel_size=(3,), stride=(1,))
  (bn4): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool4): MaxPool1d(kernel_size=4, stride=4, padding=0, dilation=1, ceil_mode=False)
  (avgPool): AvgPool1d(kernel_size=(30,), stride=(30,), padding=(0

In [ ]:
model.conv1

Conv1d(1, 32, kernel_size=(80,), stride=(4,))

In [ ]:
model.conv1

Conv1d(1, 32, kernel_size=(80,), stride=(4,))

In [ ]:
for par in model.parameters():
  par.requires_grad = False

In [ ]:
model

M5(
  (conv1): Conv1d(1, 32, kernel_size=(80,), stride=(4,))
  (bn1): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool1): MaxPool1d(kernel_size=4, stride=4, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv1d(32, 32, kernel_size=(3,), stride=(1,))
  (bn2): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool2): MaxPool1d(kernel_size=4, stride=4, padding=0, dilation=1, ceil_mode=False)
  (conv3): Conv1d(32, 64, kernel_size=(3,), stride=(1,))
  (bn3): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool3): MaxPool1d(kernel_size=4, stride=4, padding=0, dilation=1, ceil_mode=False)
  (conv4): Conv1d(64, 128, kernel_size=(3,), stride=(1,))
  (bn4): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool4): MaxPool1d(kernel_size=4, stride=4, padding=0, dilation=1, ceil_mode=False)
  (avgPool): AvgPool1d(kernel_size=(30,), stride=(30,), padding=(0

In [ ]:
model.fc1 = nn.Linear(128 , 4 , bias= True)

In [ ]:
model

M5(
  (conv1): Conv1d(1, 32, kernel_size=(80,), stride=(4,))
  (bn1): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool1): MaxPool1d(kernel_size=4, stride=4, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv1d(32, 32, kernel_size=(3,), stride=(1,))
  (bn2): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool2): MaxPool1d(kernel_size=4, stride=4, padding=0, dilation=1, ceil_mode=False)
  (conv3): Conv1d(32, 64, kernel_size=(3,), stride=(1,))
  (bn3): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool3): MaxPool1d(kernel_size=4, stride=4, padding=0, dilation=1, ceil_mode=False)
  (conv4): Conv1d(64, 128, kernel_size=(3,), stride=(1,))
  (bn4): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool4): MaxPool1d(kernel_size=4, stride=4, padding=0, dilation=1, ceil_mode=False)
  (avgPool): AvgPool1d(kernel_size=(30,), stride=(30,), padding=(0

In [ ]:
log_interval = 20
n_epoch = 7

pbar_update = 1 / (len(train_loader))
losses = []

# The transform needs to live on the same device as the model and the data.
transform = transform.to(device)
with tqdm(total=n_epoch) as pbar:
    for epoch in range(1, n_epoch + 1):
        train(model, epoch, log_interval)
        scheduler.step()

RuntimeError: ignored